#### Goal
In this analysis, the raw DOA estimation will be extracted from SSL log files across all arrays. During the recording experiment, only one source was present at all times. Since SSL log ranks DOA estimation by energy from the highest to the lowest. The first entry in each log block will be used. 

#### Procedures
Within each 5 minutes interval, take the average of first entries by each minute, e.g. 11:10-11:01, 11:01-11:02, etc. 

In [39]:
import glob
import os
import pandas as pd
import re
import json
import datetime
import time as timelib
import numpy as np
from scipy import stats

In [2]:
'''
Define functions
'''
# Extract time information of each recording from the log file
def timeExtract(filename):
    with open(filename, 'rb') as f:
        # Start counting from the last byte
        counter = 1
        # Go to the 2nd byte before the end of the last line
        f.seek(-2, 2)
        while f.read(1) != b'\n':
            f.seek(-2, 1)
            counter = counter + 1
        endTime_line = f.readline().decode()
        # Go to the 2nd byte before the end of the last second line
        f.seek(-counter - 2, 2)
        while f.read(1) != b'\n':
            f.seek(-2, 1)
        startTime_line = f.readline().decode()

    return [startTime_line, endTime_line]

# Calculate duration of each recording in microseconds
def durationinMicroseconds(filename):
    startTime = timeExtract(filename)[0].split()[2:]
    endTime = timeExtract(filename)[1].split()[2:]
    startTimeStr = startTime[0] + ' ' + startTime[1]
    endTimeStr = endTime[0] + ' ' + endTime[1]
    T1 = datetime.datetime.strptime(startTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    T2 = datetime.datetime.strptime(endTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    delta = T2 - T1
    duration = delta.seconds * 1000000 + delta.microseconds

    return duration, T1, T2


# Converts .log files into pandas dataframes
def extractData(filename, arrayNum):
    with open(filename, 'r') as f:
        text = f.read()

    # Use repex to store blocks of data
    data = re.split('(?<=})\n(?={)', text)
    # Delete the time info from the last data block
    tmp = data[-1][:(data[-1].rfind("}") + 1)]
    data[-1] = tmp

    # Store each log block into a list
    srcList = [json.loads(block)["src"] for block in data]

    # Dataframe will have columns: timestamp, time, data
    df = pd.DataFrame(
        columns=['Timestamp', 'Time', 'Time In Seconds', 'Array Number', 'X', 'Y', 'Z', 'E'])

    # Retrieve time information
    duration, startTime, endTime = durationinMicroseconds(filename)
    start_time_in_seconds = timelib.mktime(startTime.timetuple())
    t = duration / len(data) / 1000000.0

    timestamp = 1 # Timestamp in log blocks starts at 1
    ind = 0
    df_dict = {}
    for block in srcList:
        arr = np.array([block[0]["E"],block[1]["E"],block[2]["E"],block[3]["E"]])
        # Check if there is a nonzero entry, then proceed
        if arr.any(): 
            time_in_seconds = start_time_in_seconds + (timestamp - 1.0) * t
            if block[arr.argmax()]['E'] != 0:
                i = arr.argmax()
                df_dict[ind] = {"Timestamp": timestamp,
                                "Time": datetime.datetime.fromtimestamp(time_in_seconds).strftime("%B %d, %I:%M, %Y"), 
                                "Time In Seconds": time_in_seconds,
                                "Array Number": arrayNum, 
                                "X": block[i]["x"],
                                "Y": block[i]["y"], 
                                "Z": block[i]["z"], 
                                "E": block[i]["E"]}
                ind = ind + 1
        timestamp = timestamp + 1

    df = df.append(pd.DataFrame.from_dict(df_dict, "index"))
    return (df)


In [3]:
'''
Main
'''
# Obtain a list of files at 11:10 and 11:15
listOfFiles1  = glob.glob('/Users/yihanhu/Desktop/CSE4223/Data/cSSL_2020-08-19_11_10_*.log')
listOfFiles2  = glob.glob('/Users/yihanhu/Desktop/CSE4223/Data/cSSL_2020-08-19_11_15_*.log')

In [43]:
timeBins = ['August 19, 11:10, 2020',
            'August 19, 11:11, 2020',
            'August 19, 11:12, 2020',
            'August 19, 11:13, 2020',
            'August 19, 11:14, 2020']
df_dict = {}
df_SSL = pd.DataFrame(
    columns=['Time','Array Number', 
             'Mean X', 'Mean Y', 'Mean Z',  
             'STD X', 'STD Y', 'STD Z',
             'Mode X', 'Mode Y', 'Mode Z'])
ind = 0
listOfFiles = listOfFiles1
for i in range(0,len(listOfFiles)):
    arrayInd = int(listOfFiles[i][-5])
    df = extractData(listOfFiles[i],arrayInd)
    for timeMark in timeBins:
        tmp = df.loc[df['Time'] == timeMark]
        x1 = np.mean(tmp['X'])
        y1 = np.mean(tmp['Y'])
        z1 = np.mean(tmp['Z'])
        x2 = stats.mode(tmp['X'])
        y2 = stats.mode(tmp['Y'])
        z2 = stats.mode(tmp['Z'])
        x3 = np.std(tmp['X'])
        y3 = np.std(tmp['Y'])
        z3 = np.std(tmp['Z'])
        df_dict[ind] = {"Time": timeMark, 
                        "Array Number": arrayInd, 
                        "Mean X": x1,
                        "Mean Y": y1, 
                        "Mean Z": z1,
                        "STD X": x3,
                        "STD Y": y3, 
                        "STD Z": z3,
                        "Mode X": x2,
                        "Mode Y": y2,
                        "Mode Z": z2,} 
        ind = ind + 1
df_SSL = df_SSL.append(pd.DataFrame.from_dict(df_dict, "index"))

In [44]:
df_SSL = df_SSL.sort_values(by = ['Time', 'Array Number'])

In [48]:
df_SSL.loc[df_SSL['Time']=='August 19, 11:10, 2020']

,Time,Array Number,Mean X,Mean Y,Mean Z,STD X,STD Y,STD Z,Mode X,Mode Y,Mode Z
0,"August 19, 11:10, 2020",0,-0.109869,-0.169869,0.573968,0.549034,0.510499,0.259947,"([0.0], [163])","([-0.866], [76])","([0.175], [181])"
25,"August 19, 11:10, 2020",1,-0.145667,-0.298729,0.730431,0.356293,0.440342,0.187473,"([-0.271], [811])","([-0.599], [810])","([0.753], [843])"
20,"August 19, 11:10, 2020",2,-0.226128,0.179839,0.772009,0.404530,0.340599,0.202206,"([-0.326], [428])","([0.315], [295])","([0.892], [437])"
5,"August 19, 11:10, 2020",3,-0.451725,0.119296,0.434322,0.620219,0.367509,0.270927,"([-0.979], [866])","([0.108], [827])","([0.175], [994])"
15,"August 19, 11:10, 2020",4,0.126291,-0.234421,0.753014,0.364790,0.448854,0.165879,"([0.339], [496])","([-0.58], [497])","([0.74], [575])"
10,"August 19, 11:10, 2020",5,0.045916,0.135404,0.796802,0.387842,0.401868,0.181061,"([0.332], [594])","([0.387], [562])","([0.86], [824])"


In [19]:
# Examine 11:10, source point 0
timeBins = ['August 19, 11:10, 2020',
            'August 19, 11:11, 2020',
            'August 19, 11:12, 2020',
            'August 19, 11:13, 2020',
            'August 19, 11:14, 2020']
df_dict_P0 = {}
df_P0 = pd.DataFrame(columns=['Timestamp', 'Time', 'Time In Seconds', 'Array Number', 'X', 'Y', 'Z', 'E'])
ind = 0
listOfFiles = listOfFiles1
for i in range(0,len(listOfFiles)):
    arrayInd = int(listOfFiles[i][-5])
    df = extractData(listOfFiles[i],arrayInd)
    # Extract source point 0 at 11:10
    timeMark = timeBins[0] 
    tmp = df.loc[df['Time'] == timeMark]
    df_P0 = df_P0.append(tmp)

In [49]:
df_P0.loc[df_P0['Array Number'] == 2]

,Timestamp,Time,Time In Seconds,Array Number,X,Y,Z,E
0,2,"August 19, 11:10, 2020",1.597861e+09,2,0.066,0.021,0.998,0.258
1,3,"August 19, 11:10, 2020",1.597861e+09,2,0.066,0.021,0.998,0.326
2,4,"August 19, 11:10, 2020",1.597861e+09,2,0.066,0.160,0.985,0.299
3,5,"August 19, 11:10, 2020",1.597861e+09,2,-0.522,-0.381,0.763,0.194
4,6,"August 19, 11:10, 2020",1.597861e+09,2,-0.702,0.089,0.706,0.191
...,...,...,...,...,...,...,...,...
7122,7124,"August 19, 11:10, 2020",1.597861e+09,2,-0.865,0.143,0.481,0.122
7123,7125,"August 19, 11:10, 2020",1.597861e+09,2,-0.750,0.174,0.638,0.143
7124,7126,"August 19, 11:10, 2020",1.597861e+09,2,-0.865,0.143,0.481,0.130
7125,7127,"August 19, 11:10, 2020",1.597861e+09,2,-0.133,0.695,0.706,0.140
